In [1]:
import numpy as np
import csv
import pandas as pd
import re
import nltk
import random


In [2]:
# build Vocabulary
from nltk import word_tokenize

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.optimizers import SGD, Adam

In [4]:
#project_path1 = 'C:/Users/poorn/CyberSecData/'
#file_path1 = project_path1 + 'GL_Bot_New.json'
file_path = 'CyberSecControl.json'

In [5]:
# Read the json corpus data
import json
json_data = json.loads( open( file_path ).read( ) )

In [30]:
print(json_data)

{'intents': [{'tag': 'Intro', 'patterns': ['hi', 'how are you', 'is anyone there', 'hello', 'whats up', 'hey', 'yo', 'listen', 'please help me', 'i am learner from', 'i belong to', 'aiml batch', 'aifl batch', 'i am from', 'my pm is', 'blended', 'online', 'i am from', 'hey ya', 'talking to you for first time'], 'responses': ['Hello! how can i help you ?'], 'context_set': ''}, {'tag': 'Exit', 'patterns': ['thank you', 'thanks', 'cya', 'see you', 'later', 'see you later', 'goodbye', 'i am leaving', 'have a Good day', 'you helped me', 'thanks a lot', 'thanks a ton', 'you are the best', 'great help', 'too good', 'you are a good learning buddy'], 'responses': ['I hope I was able to assist you, Good Bye'], 'context_set': ''}, {'tag': 'Identity', 'patterns': ['ID.AM-1 :  Physical devices and systems within the organization are inventoried', 'ID.AM-2 :  Software platforms and applications within the organization are inventoried', 'ID.AM-3 :  Organizational communication and data flows are mappe

In [6]:
corps_voc = set()
corps_tags = []
corps_word_tag = []

{"intents": [
        {"tag": "Intro",
         "patterns": ["hi", 
                      "how are you", 
                      "online",
                      "i am from",
                      "hey ya",
                      "talking to you for first time"],
         "responses": ["Hello! how can i help you ?"],
         "context_set": ""
        }

In [7]:
# Iterate JSOn data and load contents (Corpus has Intents that has a structure
# tag - High level group
# patterns - possible inputs from user for training the model
# response -  List of responses that can be provided randomly if more than one response is available
# 

for intent_itm in json_data['intents']:
    tag = intent_itm['tag']
    corps_tags.append( tag ) #  add tag value
    for pattern in intent_itm['patterns']:
        words = word_tokenize( pattern )   #  Break the pattern row words
        corps_voc.update( words )          #  collect unique word
        corps_word_tag.append( ( words, tag ) )   
        

In [8]:
corps_voc = list( corps_voc )  # change from set to list

In [9]:
from nltk.stem import WordNetLemmatizer

In [10]:
# use WordNet alongside the NLTK module to find the meanings of words, synonyms, antonyms, and more. 
nltk.download('wordnet') 
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\poorn\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\poorn\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [11]:
# Create a list of spl characters
spl_chars = ['~', ':', "'", '+', '[', '\\', '@', '^', '{', '%', '(', '-', '"', '*', '|', ',', '&', '<', '`', '}', '.', '_', '=', ']', '!', '>', ';', '?', '#', '$', ')', '/']
wnl = WordNetLemmatizer()
corps_voc = [ wnl.lemmatize(word)  for word in corps_voc if word not in spl_chars]

In [12]:
from nltk.corpus import wordnet
wnl.lemmatize("organizations")

'organization'

In [13]:
training_data = []
output_empty = [0] * len(corps_tags)
output_empty

[0, 0, 0, 0, 0, 0, 0]

In [14]:
print(len(corps_word_tag)) # No of sentences in [patterns]
print(len(corps_voc))  #  # of words across all sentences in [patterns]
print(len(corps_tags))

144
575
7


In [15]:
for docu in corps_word_tag:
    print(docu[0], docu[1])
    

['hi'] Intro
['how', 'are', 'you'] Intro
['is', 'anyone', 'there'] Intro
['hello'] Intro
['whats', 'up'] Intro
['hey'] Intro
['yo'] Intro
['listen'] Intro
['please', 'help', 'me'] Intro
['i', 'am', 'learner', 'from'] Intro
['i', 'belong', 'to'] Intro
['aiml', 'batch'] Intro
['aifl', 'batch'] Intro
['i', 'am', 'from'] Intro
['my', 'pm', 'is'] Intro
['blended'] Intro
['online'] Intro
['i', 'am', 'from'] Intro
['hey', 'ya'] Intro
['talking', 'to', 'you', 'for', 'first', 'time'] Intro
['thank', 'you'] Exit
['thanks'] Exit
['cya'] Exit
['see', 'you'] Exit
['later'] Exit
['see', 'you', 'later'] Exit
['goodbye'] Exit
['i', 'am', 'leaving'] Exit
['have', 'a', 'Good', 'day'] Exit
['you', 'helped', 'me'] Exit
['thanks', 'a', 'lot'] Exit
['thanks', 'a', 'ton'] Exit
['you', 'are', 'the', 'best'] Exit
['great', 'help'] Exit
['too', 'good'] Exit
['you', 'are', 'a', 'good', 'learning', 'buddy'] Exit
['ID.AM-1', ':', 'Physical', 'devices', 'and', 'systems', 'within', 'the', 'organization', 'are', 'inv

In [16]:
for document in corps_word_tag:
  bag = []
  word_patterns = document[0]
  #word_patterns = [ wnl.lemmatize (word.lower())  for word in word_patterns]
  word_patterns = [ wnl.lemmatize ( x.lower() )  for x in word_patterns ]
  #print(word_patterns)
  for word in corps_voc:
    bag.append(1) if word in word_patterns else bag.append(0)
  output_row = list(output_empty)
  output_row[ corps_tags.index(document[1])] = 1
  #print(output_row)
  training_data.append( [ bag,  output_row])

In [17]:
# Prepare training data 
training_data = []
output_empty = [0] * len(corps_tags)
for document in corps_word_tag:
  bag = []
  word_patterns = document[0]
  word_patterns = [wnl.lemmatize (word.lower()) for word in word_patterns]
  #print(word_patterns)
  for word in corps_voc:
    bag.append(1) if word in word_patterns else bag.append(0)
    if word in word_patterns:
      print('match :', word)
  output_row = list (output_empty)
  output_row[corps_tags.index(document[1])] = 1
  training_data.append ([bag, output_row])

match : hi
match : are
match : how
match : you
match : anyone
match : is
match : there
match : hello
match : up
match : whats
match : hey
match : yo
match : listen
match : me
match : please
match : help
match : am
match : i
match : from
match : learner
match : to
match : belong
match : i
match : aiml
match : batch
match : aifl
match : batch
match : am
match : i
match : from
match : my
match : is
match : pm
match : blended
match : online
match : am
match : i
match : from
match : ya
match : hey
match : for
match : to
match : time
match : you
match : first
match : talking
match : you
match : thank
match : thanks
match : cya
match : see
match : you
match : later
match : see
match : later
match : you
match : goodbye
match : am
match : i
match : leaving
match : have
match : a
match : a
match : good
match : day
match : me
match : you
match : helped
match : a
match : a
match : lot
match : thanks
match : a
match : a
match : thanks
match : ton
match : are
match : you
match : best
match : the
mat

In [18]:
#print(len(training_data))
#for td in training_data:
    #print(len(td[0]),  len(td[1]))


In [19]:
#print(training_data[0][0])

In [20]:
#random.shuffle(training_data)
Data_type = str
training_data = np.array(training_data, dtype=object)
X_train = list(training_data[:, 0])
y_train = list(training_data[:, 1])

In [21]:
print(training_data.shape)
print(len(X_train[0]))
print(len(y_train[0]))

(144, 2)
575
7


In [22]:
# define the model
model = Sequential()

model.add (Dense (128, input_shape=(len(X_train[0]),), activation='relu'))
model.add (Dropout(0.5))
model.add( Dense( 64, activation='relu' ) )
model.add (Dropout(0.5))
model.add( Dense( 32, activation='relu' ) )

model.add( Dense( len(y_train[0]), activation='softmax') )

In [23]:
opt_sgd = SGD(learning_rate=0.01)
model.compile( loss='categorical_crossentropy', optimizer=opt_sgd , metrics=['accuracy'])

In [24]:
# train the model by fitting the training data
history=model.fit(np.array(X_train), np.array(y_train), epochs=200, verbose=1, batch_size=5)

Epoch 1/200
29/29 [==============================] - 0s 1ms/step - loss: 1.9307 - accuracy: 0.1875
Epoch 2/200
29/29 [==============================] - 0s 995us/step - loss: 1.9012 - accuracy: 0.2083
Epoch 3/200
29/29 [==============================] - 0s 988us/step - loss: 1.8858 - accuracy: 0.2153
Epoch 4/200
29/29 [==============================] - 0s 986us/step - loss: 1.8357 - accuracy: 0.2847
Epoch 5/200
29/29 [==============================] - 0s 983us/step - loss: 1.8444 - accuracy: 0.2500
Epoch 6/200
29/29 [==============================] - 0s 948us/step - loss: 1.7953 - accuracy: 0.2778
Epoch 7/200
29/29 [==============================] - 0s 994us/step - loss: 1.7973 - accuracy: 0.2569
Epoch 8/200
29/29 [==============================] - 0s 1ms/step - loss: 1.7769 - accuracy: 0.3056
Epoch 9/200
29/29 [==============================] - 0s 985us/step - loss: 1.7560 - accuracy: 0.2639
Epoch 10/200
29/29 [==============================] - 0s 1ms/step - loss: 1.7526 - accuracy: 0.

29/29 [==============================] - 0s 999us/step - loss: 0.4646 - accuracy: 0.8333
Epoch 83/200
29/29 [==============================] - 0s 1ms/step - loss: 0.3936 - accuracy: 0.8542
Epoch 84/200
29/29 [==============================] - 0s 1ms/step - loss: 0.4021 - accuracy: 0.8681
Epoch 85/200
29/29 [==============================] - 0s 961us/step - loss: 0.4174 - accuracy: 0.8403
Epoch 86/200
29/29 [==============================] - 0s 956us/step - loss: 0.3547 - accuracy: 0.8819
Epoch 87/200
29/29 [==============================] - 0s 1ms/step - loss: 0.3996 - accuracy: 0.8750
Epoch 88/200
29/29 [==============================] - 0s 945us/step - loss: 0.3443 - accuracy: 0.8819
Epoch 89/200
29/29 [==============================] - 0s 943us/step - loss: 0.3364 - accuracy: 0.9236
Epoch 90/200
29/29 [==============================] - 0s 966us/step - loss: 0.3438 - accuracy: 0.8819
Epoch 91/200
29/29 [==============================] - 0s 960us/step - loss: 0.2877 - accuracy: 0.9097

29/29 [==============================] - 0s 2ms/step - loss: 0.0825 - accuracy: 0.9722
Epoch 163/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1060 - accuracy: 0.9722
Epoch 164/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1567 - accuracy: 0.9653
Epoch 165/200
29/29 [==============================] - 0s 945us/step - loss: 0.1158 - accuracy: 0.9861
Epoch 166/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1122 - accuracy: 0.9583
Epoch 167/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0672 - accuracy: 1.0000
Epoch 168/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1432 - accuracy: 0.9375
Epoch 169/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0878 - accuracy: 0.9792
Epoch 170/200
29/29 [==============================] - 0s 1ms/step - loss: 0.0845 - accuracy: 0.9861
Epoch 171/200
29/29 [==============================] - 0s 1ms/step - loss: 0.1022 - accuracy: 0.9722
Ep

In [25]:
def clean_up_sentence (sentence):
  #print('Clean up ' , sentence)
  sentence_words = nltk.word_tokenize(sentence)
  sentence_words = [wnl.lemmatize(word) for word in sentence_words]
  return sentence_words


def bag_of_words (sentence):
  #print('bag_of_words ' , sentence)
  sentence_words = clean_up_sentence (sentence)
  bag = [0] * len (corps_voc)
  for w in sentence_words:
    for i, word in enumerate(corps_voc) :
      if word == w:
        bag[i] = 1
  return np.array (bag)

In [26]:
def predict_response(sentence):
  bow = bag_of_words (sentence.lower())
  res = model.predict(np.array([bow]), verbose=0)[0]
  #print(res)
  ERROR_THRESHOLD = 0.05
  results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
  #results = [[i, r] for i, r in enumerate(res) ]
  results.sort(key=lambda x: x[1], reverse=True)
  return_list= []
  for r in results:
    return_list.append({'intent': corps_tags[r[0]], 'probability' : str(r[1])})
  return return_list

In [27]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    result = []
    intro_res = " "
    #result.append('I cannot understand,  please re-phrase the question')
    for predictn in intents_list:
        tag = predictn['intent']
        #print(tag)       
        if tag == 'Exit':
            continue
        for i in list_of_intents:
            if i['tag'] == tag:
                if tag == 'Intro':
                    intro_res = random.choice(i['responses'])
                else:
                    result.append(random.choice(i['responses']))
    
    if len(result) == 0:
        result.append(intro_res)                                          
    #print(result)
    return result



In [28]:
p_r = predict_response('Hi')
print(p_r)
print(get_response (p_r, json_data))

[{'intent': 'Intro', 'probability': '0.9758312'}]
['Hello! how can i help you ?']


In [31]:
print ("Bot Running")
print("Welcome , Hi I am the Chatbot to find , preditct matching Common Security Framework using keywords, type 'Exit' to quit ")

exit_intent = 'Exit'
chat_alive = True
while chat_alive:
  message = input("")
  if message == 'Exit':
    chat_alive = False
    continue
  else:
    ints = predict_response(message.lower())
    #print('predicted....', ints)
    if( len(ints) > 1 and ints[0]['intent'] == 'Intro'):
        ints.pop(0)
    res = get_response (ints, json_data)
    chk_below_msg_printed = False
    if(len(res) == 0):
      print("Cannot comprehend,  please rephrase")
    if(ints[0]['intent'] != 'Intro'):
      print('Please refer the below Sub category and Functions for ## ',  message , ' ##')  
      print('the top one has high match probabaility')
    print(res)
    for pred_msg in res:
      pred_msg = pred_msg.strip()
      if(pred_msg == ''):
        print("Please Rephrase the search string")
      else:
        print('---- Prediction Start ------')
        print(pred_msg)
        print('---- Prediction end ------')
   

print('Good bye')  

Bot Running
Welcome , Hi I am the Chatbot to find , preditct matching Common Security Framework using keywords, type 'Exit' to quit 
agreed by organizational stakeholders
Please refer the below Sub category and Functions for ##  agreed by organizational stakeholders  ##
the top one has high match probabaility
['Please Refer NIST CSF Framework - Function Identity']
---- Prediction Start ------
Please Refer NIST CSF Framework - Function Identity
---- Prediction end ------
Exit
Good bye
